# 887. Super Egg Drop
## Description
You are given `K` eggs, and you have access to a building with `N` floors from `1` to `N`. 

Each egg is identical in function, and if an egg breaks, you cannot drop it again.

You know that there exists a floor `F` with `0 <= F <= N` such that any egg dropped at a floor higher than `F` will break, and any egg dropped at or below floor `F` will not break.

Each move, you may take an egg (if you have an unbroken one) and drop it from any floor `X` (with `1 <= X <= N`). 

Your goal is to know with certainty what the value of `F` is.

What is the minimum number of moves that you need to know with certainty what `F` is, regardless of the initial value of `F`?

### Note:
- `1 <= K <= 100`
- `1 <= N <= 10000`

### METHOD 1: Dynamic Programming with Binary Search
`MinDrop(k,n) = 1 + min( max(MinDrop(k-1, drop-1), MinDrop(k, n-drop)) for drop in range(1, n+1))`
However, since `MinDrop(k-1, drop-1)` increases while `MinDrop(k, n-drop)` decreases as `drop` increases,
the minimum of the larger one of them happens when (`MinDrop(k-1, drop-1) == MinDrop(k, n-drop)` or almost equal).
We can use binary search to find the optimal drop.
Time complexity: $O(k*N*\log N)$

In [ ]:
class Solution:
    def superEggDrop(self, K: int, N: int) -> int:
        f = {}

        def MinDrop(k, n):
            if (k,n) not in f:
                if n == 0: 
                    move = 0
                elif k == 1:
                    move = n
                else:
                    low_floor = 1
                    hi_floor = n
                    while low_floor + 1 < hi_floor:
                        mid = (low_floor + hi_floor) // 2
                        t1 = MinDrop(k-1, mid-1)
                        t2 = MinDrop(k, n-mid)
                        if t1 < t2:
                            low_floor = mid
                        elif t1 > t2:
                            hi_floor = mid
                        else:
                            low_floor = hi_floor = mid

                    move = 1 + min( max(MinDrop(k-1, drop-1), MinDrop(k, n-drop)) for drop in range(low_floor, hi_floor+1))
                
                f[k,n] = move
            return f[k,n]
        return MinDrop(K,N)

In [ ]:
### Dynamic Programming with optimal drop
### As N increases, MinDrop(k-1, drop-1) constant and MinDrop(k, n-drop) increases.
### And the optimal drop increases as N increases.
### So we keep track on the optimal drop and 
### when N increases, we start our search at the previous optimal drop instead of 1.

class Solution:
    def superEggDrop(self, K: int, N: int) -> int:
        dp = list(range(N+1))
        
        for k in range(2, K+1):
            dp2 = [0]
            drop = 1
            for floor in range(1, N+1):
                while drop < floor and max(dp[drop-1], dp2[floor-drop]) > max(dp[drop], dp2[floor-drop-1]):
                    drop += 1
                    
                dp2.append(1+max(dp[drop-1], dp2[floor-drop]))
            dp = dp2
        return dp[-1]

### METHOD 3: Mathematical method
Let `f(T,K) =` the greatest floors that can be determined with certainty for `T` tries and `K` eggs.
We want to find the smallest `T` such that `f(T,k) >= N`.
Since `f(T,K)` is an increasing function with respect to `T`, we can use binary search to find the smallest `T`.

To calculate $f(T,K)$,
Note that $f(T,K)>=n$ if and only if there exists an `drop` between 1 and $n$ such that
- f(T-1, K-1) >= drop-1
- f(T-1, K) >= n-drop

Hence `n-f(T-1, K) <= drop <= f(T-1, K-1)+1`, so `n <= f(T-1, K) + f(T-1, K-1) + 1`.
Therefore $f(T,K)$, which is the greatest possible $n$, equal to $f(T-1, K) + f(T-1, K-1) + 1$.
We can easily see that $f(0, K) = f(T, 0) = 0$ and $f(T, 1) = T$.
To solve $f(T,K) = f(T-1, K) + f(T-1, K-1) + 1$, let $g(T,K) = f(T, K) - f(T, K-1)$.
Then $g(T, K) = g(T-1, K) + g(T-1, K-1)$ with $g(T, 1) = T$. 
This means $g(T,K) = \binom(T,K)$ so $f(T,K) = \sum\limits_{i=1}^K \binom(T,i)$.

Another approach to find this is any $T$ tries is a $T$-sequence of 'success' and 'failure'.
$K$ eggs means number of 'failure' must be less than or equal to $K$.
* $T$ tries with i failure has $\binom(T,i)$ number of states, for any $0 \leq i \leq K$.
Hence $\sum\limits_{i=0}^K \binom(T,i)$ is the total number of states we can distinguish with $T$ tries and $K$ eggs. Note that this includes floor 0. This shows that $f(T,K) \leq \sum\limits_{i=1}^K \binom(T,i)$.

On the other hand, each drop at floor $i$ will show that the answer is either at most $i$ (if the egg breaks) or greater than $i$ (if the egg doesn't break). Hence, in an optimal throwing strategy, each such sequence actually maps to a unique answer.


In [ ]:
class Solution:
    def superEggDrop(self, K: int, N: int) -> int:        
        def binom(t):
            ans = 0
            r = 1
            for i in range(1, K+1):
                r *= t-i+1
                r //= i
                ans += r
                if ans >= N: break
            return ans
            
        low, high = 1, N
        while low < high:
            mid = (low + high) // 2
            if binom(mid) < N:
                low = mid + 1
            else:
                high = mid
                
        return low
        